# Análise Exploratória de Dados

In [1]:
from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
filename = 'Dados/HDR_2018_data.xlsx'
first_data = pd.read_excel(filename)

#Dados conseguidos numa outra source para categorizar os paises por regiao e income type
metadata_filename = 'Dados/HDR_2018_country_metadata.xlsx'
country_metadata = pd.read_excel(metadata_filename)

In [3]:
print(first_data.shape)
print(country_metadata.shape)

(25636, 34)
(217, 4)


In [4]:
first_data.head()

,dimension,indicator_id,indicator_name,iso3,country_name,1990,1991,1992,1993,1994,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,9999
0,Composite indices,146206,HDI rank,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.0,168.0,NaN
1,Composite indices,146206,HDI rank,ALB,Albania,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,68.0,NaN
2,Composite indices,146206,HDI rank,DZA,Algeria,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,85.0,NaN
3,Composite indices,146206,HDI rank,AND,Andorra,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,35.0,NaN
4,Composite indices,146206,HDI rank,AGO,Angola,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,147.0,NaN


In [5]:
country_metadata.head()

,Code,Long Name,Income Group,Region
0,AFG,Islamic State of Afghanistan,Low income,South Asia
1,ALB,Republic of Albania,Upper middle income,Europe & Central Asia
2,DZA,People's Democratic Republic of Algeria,Upper middle income,Middle East & North Africa
3,ASM,American Samoa,Upper middle income,East Asia & Pacific
4,AND,Principality of Andorra,High income,Europe & Central Asia


In [6]:
data = pd.merge(left=first_data, right=country_metadata, how='left', left_on='iso3', right_on='Code')

In [7]:
data.shape

(25636, 38)

In [8]:
data.head()

,dimension,indicator_id,indicator_name,iso3,country_name,1990,1991,1992,1993,1994,...,2013,2014,2015,2016,2017,9999,Code,Long Name,Income Group,Region
0,Composite indices,146206,HDI rank,AFG,Afghanistan,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,168.0,168.0,NaN,AFG,Islamic State of Afghanistan,Low income,South Asia
1,Composite indices,146206,HDI rank,ALB,Albania,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,69.0,68.0,NaN,ALB,Republic of Albania,Upper middle income,Europe & Central Asia
2,Composite indices,146206,HDI rank,DZA,Algeria,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,83.0,85.0,NaN,DZA,People's Democratic Republic of Algeria,Upper middle income,Middle East & North Africa
3,Composite indices,146206,HDI rank,AND,Andorra,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,35.0,35.0,NaN,AND,Principality of Andorra,High income,Europe & Central Asia
4,Composite indices,146206,HDI rank,AGO,Angola,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,145.0,147.0,NaN,AGO,People's Republic of Angola,Lower middle income,Sub-Saharan Africa
